<a href="https://colab.research.google.com/github/KunaCornejo/SmartGatewaySelection/blob/main/SmartSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from numpy import genfromtxt
from geopy.distance import great_circle

In [3]:
#import git
!git clone https://github.com/KunaCornejo/SmartGatewaySelection.git


Cloning into 'SmartGatewaySelection'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 36 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [4]:
ls 'SmartGatewaySelection'

c_n_ul.csv  Locaciones.csv  Matriz_Train_90.csv.zip  SmartSelection.ipynb


In [5]:
c_n_ul = genfromtxt('/content/SmartGatewaySelection/c_n_ul.csv', delimiter=',') 
c_n_ul=list(c_n_ul)

In [6]:
ubicaciones=['Panama','San Jose','Tegucigalpa','Mexico City','Monterrey','Guadalajara'\
             ,'Tijuana','La Habana','Sto. Domingo','San Salvador','San Juan',\
             'Torreon','La Paz (BC)','Veracruz','Cancun','Queretaro'\
             ,'Tuxtla (Gtz)','Cd. Juarez','San Pedro Sula','Belmopan','Kingston'\
             ,'Merida','Pto. Cabezas','Oaxaca']
ubicaciones=np.transpose([ubicaciones])
C_N_UL=np.array([c_n_ul],dtype=object) 
C_N_UL=np.transpose(C_N_UL)
C_N_UL=np.delete(C_N_UL, 15, 0) #Get rid of Chihuahua, it is not necessary
CN_UP=np.concatenate((ubicaciones,C_N_UL),axis=1) #Concatenate two lists
print(CN_UP)

[['Panama' 32.4401636]
 ['San Jose' 32.4611016]
 ['Tegucigalpa' 32.4450513]
 ['Mexico City' 32.3918091]
 ['Monterrey' 32.313151]
 ['Guadalajara' 32.3588726]
 ['Tijuana' 32.0981607]
 ['La Habana' 32.3380601]
 ['Sto. Domingo' 32.2887155]
 ['San Salvador' 32.4521009]
 ['San Juan' 32.2438352]
 ['Torreon' 32.301041]
 ['La Paz (BC)' 32.2705558]
 ['Veracruz' 32.4018877]
 ['Cancun' 32.3779865]
 ['Queretaro' 32.3736176]
 ['Tuxtla (Gtz)' 32.4294244]
 ['Cd. Juarez' 32.195777]
 ['San Pedro Sula' 32.4354086]
 ['Belmopan' 32.4215586]
 ['Kingston' 32.3572017]
 ['Merida' 32.3851709]
 ['Pto. Cabezas' 32.4314209]
 ['Oaxaca' 32.4205395]]


In [7]:
# Python code to sort the array using second element  
# of sublist Function to sort using sorted() 
def Sort(sub_li): 
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of  
    # sublist lambda has been used 
    return(sorted(sub_li, key = lambda x: x[1], reverse = True)) 
    #X[] es la columna que vamos a ordenar
# Driver Code 
sub_li = CN_UP
sub_li= np.array(Sort(sub_li))  #Just test the sorted list 
print(sub_li)

[['San Jose' 32.4611016]
 ['San Salvador' 32.4521009]
 ['Tegucigalpa' 32.4450513]
 ['Panama' 32.4401636]
 ['San Pedro Sula' 32.4354086]
 ['Pto. Cabezas' 32.4314209]
 ['Tuxtla (Gtz)' 32.4294244]
 ['Belmopan' 32.4215586]
 ['Oaxaca' 32.4205395]
 ['Veracruz' 32.4018877]
 ['Mexico City' 32.3918091]
 ['Merida' 32.3851709]
 ['Cancun' 32.3779865]
 ['Queretaro' 32.3736176]
 ['Guadalajara' 32.3588726]
 ['Kingston' 32.3572017]
 ['La Habana' 32.3380601]
 ['Monterrey' 32.313151]
 ['Torreon' 32.301041]
 ['Sto. Domingo' 32.2887155]
 ['La Paz (BC)' 32.2705558]
 ['San Juan' 32.2438352]
 ['Cd. Juarez' 32.195777]
 ['Tijuana' 32.0981607]]


In [8]:
#Unzip the .csv file
import zipfile #import ZipFile
with zipfile.ZipFile('/content/SmartGatewaySelection/Matriz_Train_90.csv.zip','r') as zip_ref:
    zip_ref.extractall('content')

In [11]:
#It is imported the predicted rain attenuation matrix obtained from Deep Learning Model 
Matrix_Arain = genfromtxt('content/Matriz_Train_90.csv', delimiter=',')
Matrix_Arain = np.delete(Matrix_Arain, 15, 0) #Get rid of Chihuahua, it is not necessary

In [12]:
#The CNIR Matrix is padding by zeros in order to load values later.
CNIR=np.zeros((len(CN_UP[:,0]),len(Matrix_Arain[0,:])),dtype=float)
Y=np.zeros((len(CN_UP[:,0]),len(Matrix_Arain[0,:])),dtype=int) 
prob=np.zeros((len(CN_UP[:,0]),1),dtype=float)  #Array 24x1

In [13]:
CI_co = 36.09  #C/I co-channel at 50 GHz (offset-paraboli-reflctor antenna simulations)
CI_adj = 38.17 #C/I adjacent at 50 Ghz
trh = 14.00    #CNIR Threshold

#CNIR [C/(N+I)] calculations for each feeder link at time-step t+1 
#(due to the predicted rain attenuation)
#Further, it calculates every feeder link availability 

for i in range(len(CN_UP[:,0])):
    CNIR[i,:]=CN_UP[i,1]-Matrix_Arain[i,0:]
    CNIR[i,:]=10*np.log10(1/(1/(np.power(10,CNIR[i,0:]/10))+1/(np.power(10,CI_co/10))\
        +1/(np.power(10,CI_adj/10))))     
    for j in range(len(CNIR[i,0:])):
        if CNIR[i,j]>=trh: 
            Y[i,j]=1
    prob[i,0]=sum(Y[i,0:])/len(Y[i,0:])
Y=np.array(Y,dtype=object)
estado = np.concatenate((ubicaciones,prob,Y),axis=1)

In [14]:
#To obtain geographic coordinates from each location in order to find
#the distance matrix D
sites = genfromtxt('/content/SmartGatewaySelection/Locaciones.csv', delimiter=',') 
sites=np.array(sites,dtype=float)
sites=np.delete(sites, 15, 0) #Get rid of Chihuahua, it is not necessary
#Creamos Matriz de Distancias entre ubicaciones
D=np.zeros((len(sites[0:,0]),len(sites[0:,0])),dtype=float)

for i in range(len(sites[0:,0])):
    for j in range(len(sites[0:,0])):
        D[i,j]=great_circle((sites[i,0],sites[i,1]),(sites[j,0],sites[j,1])).kilometers

D=np.array(D,dtype=object)
D_tag = np.concatenate((ubicaciones,D),axis=1) #Columna de encabezados
ubic = ubicaciones.tolist() #Convierte un array en una lista
ubic.insert(0,[''])
ubic = np.transpose(ubic)
D_tag = np.concatenate((ubic,D_tag),axis=0) #Fila de encabezados

#To save the distance matrix D
#np.savetxt('.../Distance.csv',D_tag,fmt='%s',delimiter=',')
print('done')

done


In [23]:
#In this routine, it is necessary to sort locations from high to low probabilities, respectively.
#Later, locations are sorted by distances > d(km) between each other.

N=8        #Number of nominal GWs (NGWs), N=4,8,12 or 16
d_trh=80   #The minimum distance in order to avoid correlated rain.

nominal_matrix=np.zeros((N,len(estado[0,0:])),dtype=object)
redundant_matrix=np.zeros(((len(estado[0:,0])-N),len(estado[0,0:])),dtype=object)

estado_ordenado = np.array(Sort(estado))
#np.savetxt('.../Ordenado.csv',estado_ordenado[:,0:2],fmt='%s',delimiter=',')

nominal_matrix=estado_ordenado[0:N,0:]
redundant_matrix=estado_ordenado[N:,0:]

filas = [' ']
contador=0
while (filas[0] == ' '):
    contador+=1
    for k in range(0,N):
        site_ini=nominal_matrix[k,0]
        i=list(D_tag[0,0:]).index(site_ini)
        print(f'\nAnalyzing the location: {site_ini}\n')
        for l in range(0,N):
            site_comp=nominal_matrix[l,0]
                    
            j=list(D_tag[0:,0]).index(site_comp)
    
            if ((k==l) & (D_tag[i,j]==0)):
                print(f'The same location is not analyzed: {site_comp}')
            elif ((k!=l) & (D_tag[i,j]>=d_trh)):
                print(f'The condition of the distance to {site_comp} is met: {D_tag[i,j]}')
            elif ((k!=l) & (D_tag[i,j]<d_trh)):
                print(f'The condition of the distance to {site_comp} is NOT met: {D_tag[i,j]}')
                filas.append(l)
        
    if len(filas)>1:
        print('\nRow Swapping')
        filas=list(dict.fromkeys(filas)) #To get rid of repeated elements in the array
        
        redundant_matrix_res=redundant_matrix #Nominal Matrix Backup
        
        for n in range(0,len(filas)-1):
            #To send until the end of the redundant matrix
            redundant_matrix=np.insert(redundant_matrix,len(estado)-N,nominal_matrix[filas[n+1],0:],axis=0)
            #Swapping the no-valid GW of the nominal matrix with the best probability of the redundant GW. 
            nominal_matrix[filas[n+1],0:] = redundant_matrix_res[n,0:]
            #Getting rid of the row of the backup matrix.
            redundant_matrix=np.delete(redundant_matrix,0, axis=0)
        filas.clear()
        filas = [' ']
    else:
        print(f'\nAll locations meet the condition, D>{d_trh} km\n')
        filas.clear()
        filas.insert(0,'done')
        
nominal_matrix=np.array(Sort(nominal_matrix))
redundant_matrix=np.array(Sort(redundant_matrix))

print(nominal_matrix)
print(redundant_matrix)


Analyzing the location: Torreon

The same location is not analyzed: Torreon
The condition of the distance to Cd. Juarez is met: 754.8701336424568
The condition of the distance to La Paz (BC) is met: 715.9003747423501
The condition of the distance to Tijuana is met: 1528.982159742854
The condition of the distance to Oaxaca is met: 1163.0943439873479
The condition of the distance to Queretaro is met: 627.8686226343513
The condition of the distance to Mexico City is met: 815.9296104834172
The condition of the distance to Guadalajara is met: 545.0972922783766

Analyzing the location: Cd. Juarez

The condition of the distance to Torreon is met: 754.8701336424568
The same location is not analyzed: Cd. Juarez
The condition of the distance to La Paz (BC) is met: 925.5643414494739
The condition of the distance to Tijuana is met: 986.6149316940383
The condition of the distance to Oaxaca is met: 1903.9311306862314
The condition of the distance to Queretaro is met: 1380.748615092518
The condition

In [24]:
#This routine counts the number of outages of the Ground Segment due to the rain events.

contador=0
corte_total=0

backup_matrix=np.zeros((6,len(estado[0,0:])),dtype=object)
outage_matrix=np.zeros((N,6),dtype=object)
outage_matrix[0:,0] = nominal_matrix[0:,0]
#To count the outages in every location
for i in range(2,len(nominal_matrix[0,2:])): 
    if (sum(nominal_matrix[0:,i]) != N):
        corte_total+=1   #Number of outages in all system
        #Switching routine
        out_array=np.array(np.where(np.isin(nominal_matrix[0:,i],0))).tolist() #To find zero values in the column array
        out_array=out_array[0] 
        k=0
        for j in range(0,len(out_array)):
            terminator=0
            while (terminator == 0):
                if redundant_matrix[k,i] == 1:
                    backup_matrix[k,i] = 1
                    backup_matrix[k,0]=redundant_matrix[k,0] #PGW Tag
                    k+=1
                    outage_matrix[out_array[j],k]+=1 #It indicates in which NGWs there is an outage
                    terminator=1
                    contador+=1  #Outage counter by intervals
                else:
                    k+=1    
print('\n')
print(sum(backup_matrix[0,2:]))
print(sum(backup_matrix[1,2:]))
print(sum(backup_matrix[2,2:]))
print(sum(backup_matrix[3,2:]))
print(sum(backup_matrix[4,2:]))
print('\n')
print(f'Outages by intervals: {contador}')
print(f'System Outages: {corte_total}')
print(f'The outage availability is: {(corte_total/len(nominal_matrix[0,2:]))*100}%') 
#Podría ser considerado probabilildad de switcheo
available=1
for i in range(0,len(nominal_matrix[0:,1])):
    available=nominal_matrix[i,1]*available
    
print(f'The total availability is: {available*100}%')
#print(f'Total: {((corte_total/len(nominal_matrix[0,2:]))*100)+(available*100)}')
print('\n')
outage_matrix=np.transpose(outage_matrix)
outage_matrix=outage_matrix[~np.all(outage_matrix == 0, axis=1)] #remove 0 rows
outage_matrix=np.transpose(outage_matrix)
print(outage_matrix)
print('\n')
backup_matrix=backup_matrix[~np.all(backup_matrix == 0, axis=1)] #remove 0 rows
print(backup_matrix)

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#To save the outage matrix with the redundant GWs that it supports it
ubic = backup_matrix[0:,0].tolist() #It converts an array into a list
ubic.insert(0,' ')
ubic=np.reshape(ubic,(-1,len(outage_matrix[0,0:]))) 
outage_matrix = np.concatenate((ubic,outage_matrix),axis=0) #Header rows

#To save into files
#np.savetxt(f'/.../out_{N}_d_{d_trh}.csv',outage_matrix,fmt='%s',delimiter=',')
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#N+P availabilities 
dim = len(nominal_matrix[0,2:]) 
available_matrix=outage_matrix
available_matrix=np.insert(outage_matrix,1,0,axis=1)
available_matrix[1:,1]=nominal_matrix[0:,1]  

for i in range(1,len(available_matrix[0:,0])):
    temp=0
    for j in range(2,len(available_matrix[0,0:])):
        available_matrix[i,j]=(sum(nominal_matrix[i-1,2:])+temp+available_matrix[i,j])/dim
        temp+=outage_matrix[i,j-1]
#To save into files
#np.savetxt(f'/.../avai_{N}_d_{d_trh}.csv',available_matrix,fmt='%s',delimiter=',')
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
print(f'done {N} GWs')    
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



5317
51
0
0
0


Outages by intervals: 5368
System Outages: 5340
The outage availability is: 1.0159817351598175%
The total availability is: 98.9831272746979%


[['Torreon' 292 1]
 ['Cd. Juarez' 412 0]
 ['La Paz (BC)' 444 0]
 ['Tijuana' 549 0]
 ['Oaxaca' 730 0]
 ['Queretaro' 793 20]
 ['Mexico City' 907 24]
 ['Guadalajara' 1190 6]]


[['Monterrey' 0 0 ... 0 0 0]
 ['Tegucigalpa' 0 0 ... 0 0 0]]
done 8 GWs
